<a href="https://colab.research.google.com/github/shubham-skv/Mega-to-Google-Drive/blob/master/Transfer_files_from_Mega_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **<font color='blue'> Mega to Google Drive - Shared by [Cheems](https://github.com/cheems) </font>**


##__Mount GDrive to /content/drive__
#####Use one of below methods. Mounting with Google Drive File Stream app is more comfortable, but it has some writing issues. So you can avoid them by using Rclone instead.

###__Mount with Google Drive File Stream__

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

###__Mount with Rclone - (More Accurate) - (Updated)__
#####**- Multiple drives including team drives can be mounted at the same runtime session**

In [ ]:
#@markdown <br><center><img src='https://rclone.org/img/logo_on_light__horizontal_color.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive_name</h3></center><br>
drive_name = "" #@param {type:"string"}
#@markdown If you want to configure this as a team drive, check the below checkbox
team_drive = False #@param {type:"boolean"}

import subprocess, time, json, os
from google.colab import output
from IPython.display import HTML, clear_output

def formatx(i, j, k):
    val=auth_response[i].split(": ")[1][1-j:-2+j+k]
    return val

rclone_status = subprocess.check_output("command -v rclone &> /dev/null",  shell=True).decode()
if rclone_status:
  pass
else:
  !curl https://rclone.org/install.sh | sudo bash
  !rclone config file
  !touch /root/.config/rclone/rclone.conf
  output.clear()
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Rclone: Installed successfully .</h2><br>"))
  time.sleep(1)
  display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Drive authentication...</h2><br>"))
  time.sleep(2)
  output.clear()

client_id="202264815644.apps.googleusercontent.com"
client_secret="X4Z3ca8xfWDb1Voo-F9a7ZxJ"
print("Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id={}&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=https://www.googleapis.com/auth/drive&response_type=code".format(client_id))
auth_code=input("Enter your authorization code: ")
auth_response = subprocess.check_output("curl -d client_id={} -d client_secret={} -d grant_type=authorization_code -d redirect_uri=urn:ietf:wg:oauth:2.0:oob -d code={} https://accounts.google.com/o/oauth2/token".format(client_id, client_secret, auth_code), shell=True).decode().split("\n")[1:6]
access_token, expires_in, refresh_token, scope, token_type = formatx(0, 0, 0), formatx(1, 1, 0), formatx(2, 0, 0), formatx(3, 0, 0), formatx(4, 0, 1)
token_in_json = "{"+", ".join("".join("".join(auth_response).split("  ")).split(","))+"}"

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Authenticated successfully.</h2><br>"))
time.sleep(2)
output.clear()

if team_drive:
  print("Fetching team drive list...")
  check_drives = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v3/drives' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  drive_list = json.loads(check_drives).get('drives')
  print("Choose a number from below.(If you want to use the 1st team drive from the below list, type 1 and press enter)")
  for i in range(len(drive_list)):
    print(i+1,"/", drive_list[i].get('name'))
    print("  \\","\""+drive_list[i].get('id')+"\"")
  selected_num = int(input("Enter the number : "))
  root_id = drive_list[selected_num-1].get('id')
else:
  print("Fetching root id...")
  check_about = subprocess.check_output('''curl \
  'https://www.googleapis.com/drive/v2/about' \
  --header 'Authorization: Bearer {}' \
  --header 'Accept: application/json' \
  --compressed'''.format(access_token),  shell=True).decode()
  root_id = json.loads(check_about).get('rootFolderId')

#read file
rfile = open(r"/root/.config/rclone/rclone.conf", "r")
rfile_content = rfile.readlines()
rfile.close()
#write file
wfile = open(r"/root/.config/rclone/rclone.conf","a+")
if len(rfile_content) > 0:
  wfile.write("\n\n")
wfile.write("[{}]\ntype = drive\nscope = drive\ntoken = {}\n".format(drive_name, token_in_json))
if team_drive:
  wfile.write("team_drive = {}".format(root_id))
else:
  wfile.write("root_folder_id = {}".format(root_id))
wfile.close()

output.clear()
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Configuration: Completed successfully</h2><br>"))
time.sleep(1)
display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Next: Mounting drive...</h2><br>"))
time.sleep(2)
output.clear()

rclone_config_name = drive_name
local_mount_location = "/content/"+drive_name+"/"

# clear nohup
open("nohup.out", 'w').close()

# unmount first
#!fusermount -u $local_mount_location 2>/dev/null

# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M &

# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()

dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

##__Transfer from Mega to GDrive__





In [11]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Transfer from Mega to GDrive</h2></center><br>
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "https://mega.nz/file/cetlHZrA#XeB-6T9l1w1s0eHY_A5MjW1c-Cx0PvrXX6CZHq3sfbw" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
#@markdown #####_*Sometimes this cell doesn't stop itself after the completion of the transfer. In case of that stop the cell manually._
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    #runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    #runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)



transfare()



TRANSFERRING ||#.........................................||(0/405 MB:   0.20 %)  
TRANSFERRING ||#.........................................||(1/405 MB:   0.37 %)  
TRANSFERRING ||#.........................................||(2/405 MB:   0.61 %)  
TRANSFERRING ||#.........................................||(4/405 MB:   1.01 %)  
TRANSFERRING ||#.........................................||(6/405 MB:   1.68 %)  
TRANSFERRING ||##.......................................||(11/405 MB:   2.80 %)  
TRANSFERRING ||##.......................................||(14/405 MB:   3.63 %)  
TRANSFERRING ||##.......................................||(15/405 MB:   3.81 %)  
TRANSFERRING ||##.......................................||(17/405 MB:   4.40 %)  
TRANSFERRING ||###......................................||(21/405 MB:   5.40 %)  
TRANSFERRING ||###......................................||(24/405 MB:   6.14 %)  
TRANSFERRING ||###......................................||(25/405 MB:   6.19 %)  
TRANSFERRING ||#

In [8]:
!sudo apt install /content/megacmd-xUbuntu_20.04_amd64.deb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'megacmd' instead of '/content/megacmd-xUbuntu_20.04_amd64.deb'
megacmd is already the newest version (1.6.3-2.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb

--2023-07-24 10:08:56--  http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
Resolving archive.ubuntu.com (archive.ubuntu.com)... 185.125.190.36, 185.125.190.39, 91.189.91.39, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|185.125.190.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1321244 (1.3M) [application/x-debian-package]
Saving to: ‘libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb’

libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  1.98MB/s    in 0.6s    

2023-07-24 10:08:57 (1.98 MB/s) - ‘libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb’ saved [1321244/1321244]



In [5]:
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2.19) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2.19) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...


In [7]:
!sudo apt-get update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://mega.nz/linux/repo/xUbuntu_20.04 ./ InRelease [2,961 B]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [731 kB]
Get:9 https://mega.nz/linux/repo/xUbuntu_20.04 ./ Packages [2,288 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [734 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,807 kB in 1s (1,388 kB/s)
Reading package lists... Done


####_Update: If you have a Mega Pro account, sign in to use its bandwidth(transfer quota)_

In [ ]:
# @markdown <center><h3>Input Your Mega ID</h3></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")
# @markdown *After signing in, use the above cell(Main Cell) to transfer files from Mega to GDrive using the transfer quota of your Mega Pro Account*

# 📚 Follow me on GitHub for more: [_@cheems_](https://github.com/cheems)
# 📦 Repo: [Github](https://github.com/cheems/Mega-to-Google-Drive)